In [6]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon

In [ ]:
# Add block-group level housing costs from H+T to 2014 parcel file

In [3]:
df = pd.read_csv(r'R:\SoundCast\Inputs\2014_esd\landuse\parcels_urbansim.txt',
                delim_whitespace=True)

In [14]:
# Load parcel geography as geoseries
geog = df.apply(lambda x: Point((float(x.xcoord_p), float(x.ycoord_p))), axis=1)
geog = gpd.GeoSeries(geog)
geog.crs = {'init': 'epsg:2285'}  # WA State Plane (north)
df['lat_lon_geog'] =  geog
df['geometry'] = geog.to_crs(epsg='2285') # Replace default geometry field with the projected epsg=2285 projection to match shapefiles
df.crs = {'init': 'epsg:2285'}

In [20]:
def spatial_join(gdf1, gdf2, keep_field, rename_field, crs):
    """Spatial join two geodataframes, left intersect with base on gdf1"""
    df = gpd.sjoin(gdf1, gdf2[['geometry',keep_field]], how="left", op='intersects')
    df = df.rename(columns={keep_field: rename_field})
    df.crs = crs
    
    return df

In [21]:
# Join parcels to block groups
block_group = gpd.GeoDataFrame.from_file(r'R:\Brice\blockgrp2010.shp')
block_group.crs = {'init' :'epsg:2285'} # WA State Plane (north)

_df = spatial_join(gdf1=df, gdf2=block_group, keep_field='GEOID10', rename_field='final_bg', crs=df.crs)

# Attach data from block group level
costs = pd.read_csv(r'C:\Users\Brice\travel-modeling\costs\htaindex_data_blkgrps_53.csv')

_df['final_bg'] = _df['final_bg'].astype('float64')
new_parcels = pd.merge(_df,costs,how='left',left_on='final_bg',right_on='blkgrp')

new_parcels.to_csv(r'R:\SoundCastDocuments\metrics\housing_transport_costs\parcel_housing_costs_2014_lodes.csv',
                  index=False)